In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
files = os.listdir('data')
files.sort()
files = files[1:-1]
files

['ebd_US-TX-013_relDec-2023.txt',
 'ebd_US-TX-019_relDec-2023.txt',
 'ebd_US-TX-029_relDec-2023.txt',
 'ebd_US-TX-091_relDec-2023.txt',
 'ebd_US-TX-171_relDec-2023.txt',
 'ebd_US-TX-187_relDec-2023.txt',
 'ebd_US-TX-259_relDec-2023.txt',
 'ebd_US-TX-265_relDec-2023.txt',
 'ebd_US-TX-325_relDec-2023.txt',
 'ebd_US-TX-493_relDec-2023.txt']

In [3]:
def clean_ebird(df_init):
    
    #make it python friendly
    df_init.columns = df_init.columns.str.lower().str.replace('\W+','_',regex=True)

    #remove unnecessary columns
    df_init = df_init.drop(columns=['global_unique_identifier','last_edited_date','taxon_concept_id',
                                    'scientific_name','subspecies_common_name','subspecies_scientific_name',
                                    'exotic_code','breeding_code','breeding_category','behavior_code',
                                    'age_sex','country','country_code','state_code','state',
                                    'county_code','iba_code','bcr_code', 'locality_id', 'locality_type',
                                    'protocol_code','protocol_type', 'project_code','effort_area_ha',
                                    'has_media','approved','reviewed','reason','trip_comments',
                                    'species_comments','unnamed_49','usfws_code','atlas_block',])

    #replace all Xs with 1s, to count the bird once
    df_init.observation_count = np.where(df_init.observation_count == 'X', 1, df_init.observation_count)
    df_init.observation_count = df_init.observation_count.astype(int)

    #find all dupe checklists
    #sort them by observation count
    #drop the duplicates, except teh first instance
    df_keep_dups = df_init [df_init.group_identifier.notnull()].sort_values('observation_count', 
                                                   ascending=False).drop_duplicates(subset=['taxonomic_order',
                                                                                            'group_identifier'])
    #remove all duplicate checklists completely
    df_no_dups = df_init [df_init.group_identifier.isnull()]


    #combine the no dups df with the first dups only df
    df = pd.concat([df_no_dups, df_keep_dups])

    #drop group identifier column, as its no longer needed
    df = df.drop('group_identifier', axis=1)

    #remove everything before 2002
    df = df [df.observation_date >= '2002-01-01']

    return df

In [4]:
clean_dfs = []

for file in files:
    print(file)
    df = pd.read_csv('data/' + file, sep='\t')
    clean_df = clean_ebird(df)
    clean_dfs.append(clean_df)

ebd_US-TX-013_relDec-2023.txt
ebd_US-TX-019_relDec-2023.txt


/var/folders/14/ft4ch4ld42z46kvx0_wfd3d40000gp/T/ipykernel_1615/71692512.py:5: DtypeWarning: Columns (11,12,13,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/' + file, sep='\t')


ebd_US-TX-029_relDec-2023.txt


/var/folders/14/ft4ch4ld42z46kvx0_wfd3d40000gp/T/ipykernel_1615/71692512.py:5: DtypeWarning: Columns (11,12,13,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/' + file, sep='\t')


ebd_US-TX-091_relDec-2023.txt
ebd_US-TX-171_relDec-2023.txt
ebd_US-TX-187_relDec-2023.txt


/var/folders/14/ft4ch4ld42z46kvx0_wfd3d40000gp/T/ipykernel_1615/71692512.py:5: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/' + file, sep='\t')


ebd_US-TX-259_relDec-2023.txt


/var/folders/14/ft4ch4ld42z46kvx0_wfd3d40000gp/T/ipykernel_1615/71692512.py:5: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/' + file, sep='\t')


ebd_US-TX-265_relDec-2023.txt
ebd_US-TX-325_relDec-2023.txt
ebd_US-TX-493_relDec-2023.txt


In [5]:
dff = pd.concat(clean_dfs)
dff = dff.reset_index(drop=True)

In [6]:
dff.head()

,taxonomic_order,category,common_name,observation_count,county,locality,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_event_identifier,duration_minutes,effort_distance_km,number_observers,all_species_reported
0,11791,species,American Kestrel,1,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1
1,233,species,Black-bellied Whistling-Duck,12,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1
2,33307,species,Brown-headed Cowbird,1,Atascosa,Atascosa Co.--CR411/412 FM99 area,28.710839,-98.214340,2002-01-11,14:35:00,obsr21142,S17226909,50.0,1.609,2.0,1
3,33325,species,Brewer's Blackbird,1,Atascosa,Atascosa Co.--CR411/412 FM99 area,28.710839,-98.214340,2002-01-11,14:35:00,obsr21142,S17226909,50.0,1.609,2.0,1
4,8416,species,Barn Owl,1,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1


In [7]:
dff.shape

(3626211, 16)

In [8]:
dff.county.value_counts()

Bexar        1765735
Guadalupe     396980
Kendall       372021
Comal         311381
Kerr          283787
Bandera       232628
Wilson         77269
Medina         75427
Gillespie      72553
Atascosa       38430
Name: county, dtype: int64

In [9]:
dff.to_csv('ebird_all_BAS.csv')

In [10]:
dff

,taxonomic_order,category,common_name,observation_count,county,locality,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_event_identifier,duration_minutes,effort_distance_km,number_observers,all_species_reported
0,11791,species,American Kestrel,1,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1
1,233,species,Black-bellied Whistling-Duck,12,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1
2,33307,species,Brown-headed Cowbird,1,Atascosa,Atascosa Co.--CR411/412 FM99 area,28.710839,-98.214340,2002-01-11,14:35:00,obsr21142,S17226909,50.0,1.609,2.0,1
3,33325,species,Brewer's Blackbird,1,Atascosa,Atascosa Co.--CR411/412 FM99 area,28.710839,-98.214340,2002-01-11,14:35:00,obsr21142,S17226909,50.0,1.609,2.0,1
4,8416,species,Barn Owl,1,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3626206,34060,species,Indigo Bunting,1,Wilson,Floresville River Park,29.112988,-98.171639,2019-05-03,15:00:00,obsr280995,S55963999,60.0,3.219,13.0,1
3626207,34060,species,Indigo Bunting,1,Wilson,Stockdale WTP,29.231437,-97.977619,2019-05-03,11:07:00,obsr172206,S55715622,27.0,NaN,13.0,1
3626208,5798,species,Killdeer,1,Wilson,Jackson Nature Pk (Wilson Co.),29.219489,-98.009613,2019-05-03,11:54:00,obsr172206,S55724322,113.0,2.253,13.0,1
3626209,5798,species,Killdeer,1,Wilson,Floresville River Park,29.112988,-98.171639,2019-05-03,15:06:00,obsr30058,S55827344,60.0,1.609,13.0,1


In [11]:
dff['month'] = dff.observation_date.str[5:7]
dff['year'] = dff.observation_date.str[:4]

In [12]:
dff.year.unique()

array(['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022', '2023'], dtype=object)

In [13]:
dff.year.nunique()

22

In [14]:
round(dff [dff.county == 'Bexar'].groupby('month').sampling_event_identifier.nunique() / dff.year.nunique(),0)

month
01    680.0
02    604.0
03    685.0
04    809.0
05    692.0
06    366.0
07    329.0
08    348.0
09    437.0
10    439.0
11    440.0
12    530.0
Name: sampling_event_identifier, dtype: float64

In [15]:
dff

,taxonomic_order,category,common_name,observation_count,county,locality,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_event_identifier,duration_minutes,effort_distance_km,number_observers,all_species_reported,month,year
0,11791,species,American Kestrel,1,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1,01,2002
1,233,species,Black-bellied Whistling-Duck,12,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1,01,2002
2,33307,species,Brown-headed Cowbird,1,Atascosa,Atascosa Co.--CR411/412 FM99 area,28.710839,-98.214340,2002-01-11,14:35:00,obsr21142,S17226909,50.0,1.609,2.0,1,01,2002
3,33325,species,Brewer's Blackbird,1,Atascosa,Atascosa Co.--CR411/412 FM99 area,28.710839,-98.214340,2002-01-11,14:35:00,obsr21142,S17226909,50.0,1.609,2.0,1,01,2002
4,8416,species,Barn Owl,1,Atascosa,Peeler Tank,28.735340,-98.435998,2002-01-05,NaN,obsr88598,S5554748,NaN,NaN,NaN,1,01,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3626206,34060,species,Indigo Bunting,1,Wilson,Floresville River Park,29.112988,-98.171639,2019-05-03,15:00:00,obsr280995,S55963999,60.0,3.219,13.0,1,05,2019
3626207,34060,species,Indigo Bunting,1,Wilson,Stockdale WTP,29.231437,-97.977619,2019-05-03,11:07:00,obsr172206,S55715622,27.0,NaN,13.0,1,05,2019
3626208,5798,species,Killdeer,1,Wilson,Jackson Nature Pk (Wilson Co.),29.219489,-98.009613,2019-05-03,11:54:00,obsr172206,S55724322,113.0,2.253,13.0,1,05,2019
3626209,5798,species,Killdeer,1,Wilson,Floresville River Park,29.112988,-98.171639,2019-05-03,15:06:00,obsr30058,S55827344,60.0,1.609,13.0,1,05,2019


In [16]:
df_species = dff [dff.category == 'species']

In [17]:
for county in df_species.county.unique():
    print()
    print(county)
    print('-----')
    subset = df_species [df_species.county == county]
    
    all_species = []
    
    for year in subset.year.unique():
        this_year_species = subset [subset.year == year].common_name.unique()
        all_species.extend(this_year_species)

    all_species = pd.Series(all_species)
    year_all = all_species.value_counts() [all_species.value_counts() == 22].index
    year_one = all_species.value_counts() [all_species.value_counts() == 1].index
    
#     for year in subset.year.unique():
#         this_year_species = subset [subset.year == year].common_name.unique()
#         print(f'{year}')
#         year_one_species = np.intersect1d(year_one, this_year_species)
#         if len(year_one_species) > 0:
#             for species in year_one_species:
#                 print(f'  {species}')
    print('all years')
    print(year_all)


Atascosa
-----
all years
Index([], dtype='object')

Bandera
-----
all years
Index(['American Kestrel', 'Blue Grosbeak', 'Northern Mockingbird',
       'Nashville Warbler', 'Louisiana Waterthrush', 'Lincoln's Sparrow',
       'Lesser Goldfinch', 'Lark Sparrow', 'Indigo Bunting', 'House Wren',
       'Hermit Thrush', 'Green Kingfisher', 'Golden-fronted Woodpecker',
       'Field Sparrow', 'Eastern Wood-Pewee', 'Eastern Screech-Owl',
       'Cooper's Hawk', 'Cliff Swallow', 'Clay-colored Sparrow',
       'Cedar Waxwing', 'Canyon Towhee', 'Bushtit',
       'Black-throated Green Warbler', 'Belted Kingfisher',
       'Yellow-throated Warbler', 'Northern Rough-winged Swallow',
       'Purple Martin', 'American Robin', 'Zone-tailed Hawk',
       'Yellow-billed Cuckoo', 'Woodhouse's Scrub-Jay', 'Wilson's Warbler',
       'Wild Turkey', 'White-crowned Sparrow', 'Cave Swallow',
       'Vermilion Flycatcher', 'Summer Tanager', 'Olive Sparrow',
       'Spotted Towhee', 'Sharp-shinned Hawk', 'Sciss

all years
Index([], dtype='object')
